# **Extracting Features**
#### This exercise covers transforming the features with 1-of-k encoding also known as One Hot encoding. 
#### ** This exercise will cover: **
+  ####*Part 1:* Featurize categorical data using 1-of-k encoding (One Hot encoding)
+  ####*Part 2:* Construct an 1-of-k encoding dictionary
 
#### Note that, for reference, you can look up the details of the relevant Spark methods in [Spark's Python API](https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD) and the relevant NumPy methods in the [NumPy Reference](http://docs.scipy.org/doc/numpy/reference/index.html)

### ** Part 1: Featurize categorical data using 1-of-k encoding **

#### ** (1a) 1-of-k encoding **
#### Categorical features refer to variables that can take one of a set of possible states at any given time.We would like to develop code to convert categorical features to numerical ones. In this exercise, we will work with a sample unlabeled dataset with four data points, with each data point representing a city. The first feature indicates the name of city (Beijing, Paris, Londre, New York); the second feature describes the temperature of the city (33., 15.); and the third (optional) feature describes how the city is polluted (very much, a little).
#### In an 1-of-k approach, we want to represent each tuple of `(featureID, category)` via its own binary feature.  We can do this in Python by creating a dictionary that maps each tuple to a distinct integer, where the integer corresponds to a binary feature. To start, manually enter the entries in the OHE dictionary associated with the sample dataset by mapping the tuples to consecutive integers starting from zero,  ordering the tuples first by featureID and next by category.
#### Later in this workshop, we'll use OHE dictionaries to transform data points into compact lists of features that can be used in machine learning algorithms.

In [43]:
# Data for manual OHE
# Note: the first data point does not include any value for the optional third feature
cityOne = [(0, 'Beijing'), (1, '33.'), (2, 'very much')]
cityTwo = [(0, 'Paris'), (1, '15.')]
cityThree =  [(0, 'London'), (1, '15.'), (2, 'a little')]
cityFour =  [(0, 'New York'), (1, '10.'), (2, 'a little')]
sampleDataRDD = sc.parallelize([cityOne, cityTwo, cityThree,cityFour])

In [44]:
# TODO: Replace <FILL IN> with appropriate code
cityOHEDict = {}
cityOHEDict[(0,'Beijing')] = 0
cityOHEDict[(0,'Paris')] = 1
cityOHEDict[(0,'London')] = 2
cityOHEDict[(0,'New York')] = 3
cityOHEDict[(1, '33.')] = 4
cityOHEDict[(1, '15.')] = 5
cityOHEDict[(1, '10.')] = 6
cityOHEDict[(2, 'very much')] = 7
cityOHEDict[(2, 'a little')] = 8

#### ** (1b) Sparse vectors **
#### Data points can typically be represented with a small number of non-zero 1-of-k encoding features relative to the total number of features that occur in the dataset.  By leveraging this sparsity and using sparse vector representations of 1-of-k encoding data, we can reduce storage and computational burdens.  Below are a few sample vectors represented as dense numpy arrays.  Use [SparseVector](https://spark.apache.org/docs/latest/api/python/pyspark.mllib.html#pyspark.mllib.linalg.SparseVector) to represent them in a sparse fashion, and verify that both the sparse and dense representations yield the same results when computing [dot products](http://en.wikipedia.org/wiki/Dot_product) (we will later use MLlib to train classifiers via gradient descent, and MLlib will need to compute dot products between SparseVectors and dense parameter vectors).
#### Use `SparseVector(size, *args)` to create a new sparse vector where size is the length of the vector and args is either a dictionary, a list of (index, value) pairs, or two separate arrays of indices and values (sorted by index).  You'll need to create a sparse vector representation of each dense vector `aDense` and `bDense`.

In [45]:
import numpy as np
from pyspark.mllib.linalg import SparseVector

In [46]:
# TODO: Replace <FILL IN> with appropriate code
aDense = np.array([0., 3., 0., 4.])
aSparse = SparseVector(len(aDense), [1,3],[3.,4.])

bDense = np.array([0., 0., 0., 1.])
bSparse = SparseVector(len(bDense), [3],[1.])

w = np.array([0.4, 3.1, -1.4, -.5])
print aDense.dot(w)
print aSparse.dot(w)
print bDense.dot(w)
print bSparse.dot(w)

7.3
7.3
-0.5
-0.5


In [47]:
# TEST Sparse Vectors (1b)
from test_helper import Test
Test.assertTrue(isinstance(aSparse, SparseVector), 'aSparse needs to be an instance of SparseVector')
Test.assertTrue(isinstance(bSparse, SparseVector), 'aSparse needs to be an instance of SparseVector')
Test.assertTrue(aDense.dot(w) == aSparse.dot(w),
                'dot product of aDense and w should equal dot product of aSparse and w')
Test.assertTrue(bDense.dot(w) == bSparse.dot(w),
                'dot product of bDense and w should equal dot product of bSparse and w')

1 test passed.
1 test passed.
1 test passed.
1 test passed.


#### **(1c) 1-of-k encoding features as sparse vectors **
#### Now let's see how we can represent the 1-of-k features for points in our sample dataset.  Using the mapping defined by the OHE dictionary from Part (1a), manually define 1-of-k features for the three sample data points using SparseVector format.  Any feature that occurs in a point should have the value 1.0.  For example, the `DenseVector` for a point with features 1 and 5 would be `[0.0, 1.0, 0.0, 0.0, 0.0,1.0, 0.0, 0.0,0.0,0.0]`.

In [48]:
# Reminder of the sample features
# cityOne = [(0, 'Beijing'), (1, '33.'), (2, 'very much')]
# cityTwo = [(0, 'Paris'), (1, '15.')]
# cityThree =  [(0, 'London'), (1, '15.'), (2, 'a little')]
# cityFour =  [(0, 'New York'), (1, '10.'), (2, 'a little')]

In [49]:
# TODO: Replace <FILL IN> with appropriate code
cityOneOHEFeaturesManual = SparseVector(9, [0,4,7],[1.,1.,1.])
cityTwoOHEFeaturesManual =  SparseVector(9, [1,5],[1.,1.])
cityThreeOHEFeaturesManual =  SparseVector(9, [2,5,8],[1.,1.,1.])
cityFourOHEFeaturesManual =  SparseVector(9, [3,6,8],[1.,1.,1.])
print cityOneOHEFeaturesManual
print cityTwoOHEFeaturesManual

(9,[0,4,7],[1.0,1.0,1.0])
(9,[1,5],[1.0,1.0])


In [50]:
# TEST OHE Features as sparse vectors (1c)
Test.assertTrue(isinstance(cityOneOHEFeatures, SparseVector),
                'cityOneOHEFeatures needs to be a SparseVector')
Test.assertTrue(isinstance(cityTwoOHEFeatures, SparseVector),
                'cityTwoOHEFeatures needs to be a SparseVector')
Test.assertTrue(isinstance(cityThreeOHEFeatures, SparseVector),
                'cityThreeOHEFeatures needs to be a SparseVector')
Test.assertTrue(isinstance(cityFourOHEFeatures, SparseVector),
                'cityFourOHEFeatures needs to be a SparseVector')

1 test passed.
1 test passed.
1 test passed.
1 test passed.


#### **(1d) Define a OHE function **
#### Next we will use the OHE dictionary from Part (1a) to programatically generate OHE features from the original categorical data.  First write a function called `oneHotEncoding` that creates OHE feature vectors in `SparseVector` format.  Then use this function to create OHE features for the first sample data point and verify that the result matches the result from Part (1c).

In [53]:
# TODO: Replace <FILL IN> with appropriate code
def oneHotEncoding(rawFeats, OHEDictionary):
    """Produce a 1-of-k encoding from a list of features and an 1-of-k dictionary.

    Note:
        You should ensure that the indices used to create a SparseVector are sorted.

    Args:
        rawFeats (list of (int, str)): The features corresponding to a single observation.  Each
            feature consists of a tuple of featureID and the feature's value. (e.g. sampleOne)
        OHEDictionary (dict): A mapping of (featureID, value) to unique integer.

    Returns:
        SparseVector: A SparseVector of length numOHEFeats with indicies equal to the unique
            identifiers for the (featureID, value) combinations that occur in the observation and
            with values equal to 1.0.
    """
    return SparseVector(len(OHEDictionary),sorted([(OHEDictionary[x],1.0) for x in rawFeats]))

# Run oneHotEnoding on sampleOne
print cityOHEDict
cityOneOHEFeatures = oneHotEncoding(cityOne, cityOHEDict)

print cityOneOHEFeatures

{(0, 'Beijing'): 0, (0, 'London'): 2, (0, 'New York'): 3, (1, '10.'): 6, (2, 'a little'): 8, (1, '33.'): 4, (2, 'very much'): 7, (0, 'Paris'): 1, (1, '15.'): 5}
(9,[0,4,7],[1.0,1.0,1.0])


In [54]:
# TEST Define an OHE Function (1d)
Test.assertTrue(cityOneOHEFeatures == cityOneOHEFeaturesManual,
                'cityOneOHEFeatures should equal cityOneOHEFeaturesManual')
Test.assertEquals(cityOneOHEFeatures, SparseVector(9,[0,4,7],[1.0,1.0,1.0]),
                  'incorrect value for cityOneOHEFeatures')
Test.assertEquals(oneHotEncoding([(0, 'Beijing'), (1, '33.'),(2, 'very much')], cityOHEDict), 
                  SparseVector(9, [0,4,7], [1.0,1.0,1.0]),
                  'incorrect definition for oneHotEncoding')

1 test passed.
1 test passed.
1 test passed.


#### **(1e) Apply OHE to a dataset **
#### Finally, use the function from Part (1d) to create OHE features for all 4 data points in the sample dataset.

In [55]:
# TODO: Replace <FILL IN> with appropriate code
sampleOHEData = sampleDataRDD.map(lambda x: oneHotEncoding(x, cityOHEDict))
print sampleOHEData.collect()

[SparseVector(9, {0: 1.0, 4: 1.0, 7: 1.0}), SparseVector(9, {1: 1.0, 5: 1.0}), SparseVector(9, {2: 1.0, 5: 1.0, 8: 1.0}), SparseVector(9, {3: 1.0, 6: 1.0, 8: 1.0})]


In [56]:
# TEST Apply OHE to a dataset (1e)
sampleOHEDataValues = sampleOHEData.collect( )
Test.assertTrue(len(sampleOHEDataValues) == 4, 'sampleOHEData should have four elements')
Test.assertEquals(sampleOHEDataValues[0], SparseVector(9, {0: 1.0, 4: 1.0, 7: 1.0}),
                  'incorrect OHE for first sample')
Test.assertEquals(sampleOHEDataValues[1], SparseVector(9, {1: 1.0, 5: 1.0}),
                  'incorrect OHE for second sample')
Test.assertEquals(sampleOHEDataValues[2], SparseVector(9, {2: 1.0, 5: 1.0, 8: 1.0}),
                  'incorrect OHE for third sample')

1 test passed.
1 test passed.
1 test passed.
1 test passed.


### ** Part 2: Construct an OHE dictionary **

#### **(2a) Pair RDD of `(featureID, category)` **
#### To start, create an RDD of distinct `(featureID, category)` tuples. In our sample dataset, the 9 items in the resulting RDD are `(0, 'Beijing')`, `(0, 'Paris')`, `(0, 'London')`, `(0, 'New York')`,`(1, '33.')`, `(1, '15.')`, `(1, '10.')`, `(2, 'a little')`, `(2, 'very much')`. Notably `'15.'` appears twice in the dataset but only contributes one item to the RDD: `(1, '15')`.  Use [flatMap](https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.flatMap) and [distinct](https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.distinct).

In [60]:
# TODO: Replace <FILL IN> with appropriate code
sampleDistinctFeats = (sampleDataRDD
                       .flatMap(lambda x: x).distinct())
print sorted(sampleDistinctFeats.collect())

[(0, 'Beijing'), (0, 'London'), (0, 'New York'), (0, 'Paris'), (1, '10.'), (1, '15.'), (1, '33.'), (2, 'a little'), (2, 'very much')]


In [61]:
# TEST Pair RDD of (featureID, category) (2a)
Test.assertEquals(sorted(sampleDistinctFeats.collect()),
                  [(0, 'Beijing'), (0, 'London'), (0, 'New York'), (0, 'Paris'), 
                   (1, '10.'), (1, '15.'), (1, '33.'), (2, 'a little'), (2, 'very much')],
                  'incorrect value for sampleDistinctFeats')

1 test passed.


#### ** (2b) OHE Dictionary from distinct features **
#### Next, create an `RDD` of key-value tuples, where each `(featureID, category)` tuple in `sampleDistinctFeats` is a key and the values are distinct integers ranging from 0 to (number of keys - 1).  Then convert this `RDD` into a dictionary, which can be done using the `collectAsMap` action.  Note that there is no unique mapping from keys to values, as all we require is that each `(featureID, category)` key be mapped to a unique integer between 0 and the number of keys.  In this exercise, any valid mapping is acceptable.  Use [zipWithIndex](https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.zipWithIndex) followed by [collectAsMap](https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.collectAsMap).

In [62]:
# TODO: Replace <FILL IN> with appropriate code
sampleOHEDict = (sampleDistinctFeats
                                    .zipWithIndex().collectAsMap())
print sampleOHEDict

{(0, 'Beijing'): 0, (0, 'London'): 1, (0, 'New York'): 2, (1, '10.'): 3, (2, 'a little'): 4, (1, '33.'): 5, (2, 'very much'): 6, (0, 'Paris'): 7, (1, '15.'): 8}


In [68]:
# TEST OHE Dictionary from distinct features (2b)
print sorted(sampleOHEDict.keys())
Test.assertEquals(sorted(sampleOHEDict.keys()),
                  [(0, 'Beijing'), (0, 'London'), (0, 'New York'), (0, 'Paris'), (1, '10.'), (1, '15.'), 
                   (1, '33.'), (2, 'a little'), (2, 'very much')],
                  'sampleOHEDict has unexpected keys')
Test.assertEquals(sorted(sampleOHEDict.values()), range(9), 'sampleOHEDict has unexpected values')

[(0, 'Beijing'), (0, 'London'), (0, 'New York'), (0, 'Paris'), (1, '10.'), (1, '15.'), (1, '33.'), (2, 'a little'), (2, 'very much')]
1 test passed.
1 test passed.


#### **(2c) Automated creation of an OHE dictionary **
#### Now use the code from Parts (2a) and (2b) to write a function that takes an input dataset and outputs an OHE dictionary.  Then use this function to create an OHE dictionary for the sample dataset, and verify that it matches the dictionary from Part (2b).

In [69]:
# TODO: Replace <FILL IN> with appropriate code
def createOneHotDict(inputData):
    """Creates a one-hot-encoder dictionary based on the input data.

    Args:
        inputData (RDD of lists of (int, str)): An RDD of observations where each observation is
            made up of a list of (featureID, value) tuples.

    Returns:
        dict: A dictionary where the keys are (featureID, value) tuples and map to values that are
            unique integers.
    """
    
    DistinctFeats = (inputData
                              .flatMap(lambda x: x).distinct())
    
    OHEDict = (DistinctFeats
                            .zipWithIndex().collectAsMap())
    
    return OHEDict

sampleOHEDictAuto = createOneHotDict(sampleDataRDD)
print sampleOHEDictAuto

{(0, 'Beijing'): 0, (0, 'London'): 1, (0, 'New York'): 2, (1, '10.'): 3, (2, 'a little'): 4, (1, '33.'): 5, (2, 'very much'): 6, (0, 'Paris'): 7, (1, '15.'): 8}


In [34]:
# TEST Automated creation of an OHE dictionary (2c)
Test.assertEquals(sorted(sampleOHEDictAuto.keys()),
                  [(0, 'Beijing'), (0, 'London'), (0, 'New York'), (0, 'Paris'), (1, '10.'), (1, '15.'), 
                   (1, '33.'), (2, 'a little'), (2, 'very much')],
                  'sampleOHEDictAuto has unexpected keys')
Test.assertEquals(sorted(sampleOHEDictAuto.values()), range(9),
                  'sampleOHEDictAuto has unexpected values')

1 test passed.
1 test passed.
